# Part IX — Deployment and Production Operations  
## 38. CI/CD (Automation That Professionals Expect)

CI/CD is how real teams ship safely and repeatedly:

- **CI (Continuous Integration)** automatically checks every change:
  - lint/format
  - tests
  - migrations
  - security checks (optional but common)
- **CD (Continuous Delivery/Deployment)** automates releases:
  - build artifact/container
  - run migrations
  - deploy web + workers
  - run smoke tests
  - rollback if necessary

This chapter gives you an industry-standard pipeline you can use on any platform.
We’ll use **GitHub Actions** as an example because it’s common, but the concepts
apply to GitLab CI, CircleCI, Jenkins, etc.

---

## 38.0 Learning Outcomes

By the end, you should be able to:

1. Define a CI pipeline that:
   - installs dependencies
   - runs Ruff, Black (check mode), tests, coverage
   - runs `makemigrations --check --dry-run`
   - runs `migrate` on a fresh DB
2. Run CI against PostgreSQL (not SQLite) to match production.
3. Add a minimal security scanning step (dependency audit) without noise.
4. Build and publish a Docker image (optional but industry common).
5. Design a CD workflow:
   - staged deploys (staging → production)
   - migrations and collectstatic steps
   - smoke checks (`/readyz`)
6. Understand secrets management in CI.
7. Prevent “works locally but fails in CI” issues with a consistent workflow.

---

## 38.1 What CI/CD Should Enforce (Professional Gatekeeping)

### 38.1.1 Minimum CI checks (baseline)
- Lint: `ruff check .`
- Format: `black --check .`
- Tests: `pytest` or `python manage.py test`
- Migrations:
  - `makemigrations --check --dry-run`
  - `migrate` to ensure migration graph applies
- Optional but recommended:
  - coverage threshold
  - query-count regression tests
  - static type checks (mypy/pyright) later
  - security checks (pip-audit) later

### 38.1.2 Why CI must run migrations
Because one of the most common production failure modes is:
- code deployed but schema mismatched
- migrations missing or broken
- migration file not committed

CI must catch that.

---

## 38.2 Prepare Your Repo for CI (Make Local Commands Match CI)

Create a `Makefile` (optional but very helpful):

```makefile
.PHONY: lint format test ci

lint:
	python -m ruff check .

format:
	python -m black .

test:
	python -m pytest

ci:
	python -m ruff check .
	python -m black . --check
	python -m pytest --cov=. --cov-report=term-missing
	python manage.py makemigrations --check --dry-run
```

### Why this is useful
- Developers run `make ci` before pushing.
- CI runs the same commands.
- This reduces “it fails only on CI” surprises.

If you don’t like Makefiles, use a `scripts/ci.sh` shell script.

---

## 38.3 CI with PostgreSQL and Redis (Match Production)

Your app likely depends on:
- PostgreSQL (production)
- Redis (Celery broker, Channels channel layer, caching)

CI should use services:
- Postgres container
- Redis container

Even if you skip Redis in early CI, make sure DB tests run against Postgres.

---

## 38.4 GitHub Actions: Full CI Workflow (Recommended Baseline)

Create `.github/workflows/ci.yml`:

```yaml
name: CI

on:
  push:
    branches: ["main"]
  pull_request:

jobs:
  test:
    runs-on: ubuntu-latest

    services:
      postgres:
        image: postgres:16-alpine
        env:
          POSTGRES_DB: django_app
          POSTGRES_USER: django
          POSTGRES_PASSWORD: django
        ports:
          - 5432:5432
        options: >-
          --health-cmd="pg_isready -U django -d django_app"
          --health-interval=10s
          --health-timeout=5s
          --health-retries=5

      redis:
        image: redis:7-alpine
        ports:
          - 6379:6379
        options: >-
          --health-cmd="redis-cli ping"
          --health-interval=10s
          --health-timeout=5s
          --health-retries=5

    env:
      DJANGO_SETTINGS_MODULE: config.settings.ci
      DJANGO_SECRET_KEY: "ci-secret"
      DJANGO_DEBUG: "false"
      DJANGO_ALLOWED_HOSTS: "testserver,localhost,127.0.0.1"
      DJANGO_CSRF_TRUSTED_ORIGINS: "http://localhost"
      SITE_URL: "http://localhost"

      POSTGRES_DB: django_app
      POSTGRES_USER: django
      POSTGRES_PASSWORD: django
      POSTGRES_HOST: "127.0.0.1"
      POSTGRES_PORT: "5432"

      CELERY_BROKER_URL: "redis://127.0.0.1:6379/0"
      CELERY_RESULT_BACKEND: "redis://127.0.0.1:6379/1"

    steps:
      - name: Checkout
        uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.12"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          python -m pip install -r requirements.txt
          python -m pip install -r requirements-dev.txt

      - name: Lint (Ruff)
        run: python -m ruff check .

      - name: Format check (Black)
        run: python -m black . --check

      - name: Django checks
        run: |
          python manage.py check --deploy
          python manage.py makemigrations --check --dry-run

      - name: Migrate
        run: python manage.py migrate

      - name: Run tests
        run: python -m pytest --cov=. --cov-report=term-missing
```

### Explanation (why this is structured like industry CI)

- Services provide Postgres and Redis.
- Separate steps:
  - install
  - lint
  - format check
  - migrations check
  - migrate
  - tests with coverage
- Environment variables emulate production-like configuration but with CI-safe values.

---

## 38.5 Add a CI Settings Module (Avoid Polluting Prod Settings)

Create `config/settings/ci.py`:

```python
from .base import *

DEBUG = False

ALLOWED_HOSTS = ["testserver", "localhost", "127.0.0.1"]

# Use Postgres env vars (same as prod style)
DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.postgresql",
        "NAME": os.environ.get("POSTGRES_DB", "django_app"),
        "USER": os.environ.get("POSTGRES_USER", "django"),
        "PASSWORD": os.environ.get("POSTGRES_PASSWORD", "django"),
        "HOST": os.environ.get("POSTGRES_HOST", "127.0.0.1"),
        "PORT": os.environ.get("POSTGRES_PORT", "5432"),
    }
}

# Faster password hashing in tests (industry standard trick)
PASSWORD_HASHERS = ["django.contrib.auth.hashers.MD5PasswordHasher"]

# Celery tasks run eagerly in CI so tests don’t need a worker
CELERY_TASK_ALWAYS_EAGER = True
CELERY_TASK_EAGER_PROPAGATES = True

# Channel layer can be in-memory in CI unless you test Redis channels explicitly
CHANNEL_LAYERS = {
    "default": {"BACKEND": "channels.layers.InMemoryChannelLayer"}
}

EMAIL_BACKEND = "django.core.mail.backends.locmem.EmailBackend"
```

### Why eager Celery in CI is standard
- avoids running real Celery worker in CI
- keeps tests deterministic
- still tests task logic and idempotency

---

## 38.6 Dependency Security Scanning (Minimal, Useful, Not Noisy)

Add `pip-audit` to dev deps:

```text
pip-audit
```

Then in CI:

```yaml
      - name: Dependency audit (pip-audit)
        run: |
          python -m pip install pip-audit
          pip-audit -r requirements.txt
```

**Industry note:** vulnerability scanning can be noisy. Start with “report-only”
and then enforce once your team can handle updates.

---

## 38.7 Build Artifacts: Docker Image (Modern Industry Standard)

If you deploy with Docker, CI should build the image.

Add a job:

```yaml
  build:
    runs-on: ubuntu-latest
    needs: [test]
    steps:
      - uses: actions/checkout@v4

      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3

      - name: Build image
        run: docker build -t myapp:${{ github.sha }} .
```

To publish to a registry (GitHub Container Registry, Docker Hub), you would:
- login with secrets
- push image tags
This depends on where you deploy.

---

## 38.8 CD (Deployment) Patterns (Platform-Agnostic)

### 38.8.1 Golden rule: deploy to staging first
Even for small teams:
- staging catches CORS/CSRF/TLS issues
- staging tests migrations on real-like DB
- staging verifies worker processes and Redis connectivity

### 38.8.2 Common CD steps
1. Build artifact (container)
2. Deploy to staging
3. Run migrations (release job)
4. Run smoke tests (curl /readyz)
5. Promote to production
6. Monitor (error rate, latency)

### 38.8.3 Where migrations run (industry practice)
Three common approaches:

- Run migrations as part of deploy (release step)
- Run migrations as a separate job that runs before web restarts
- Use a dedicated migration runner service in Kubernetes

**Rule:** only one migration runner at a time.

---

## 38.9 Secrets in CI (How to Do It Safely)

### 38.9.1 CI secrets must not be in repo
Use:
- GitHub Actions secrets
- GitLab CI variables
- Vault/secret manager

### 38.9.2 Least privilege
- CI should have minimal permissions:
  - read repo
  - publish artifacts if needed
  - deploy keys only for staging/prod deploy job (not for PR jobs)

### 38.9.3 Separate secrets per environment
- staging secrets differ from prod
- never use prod secrets in PR builds

---

## 38.10 CI Performance Tips (Keep It Fast)

- Cache pip downloads (GitHub Actions supports caching)
- Avoid rebuilding everything unnecessarily
- Keep test DB setup efficient
- Use faster password hashers in CI settings (already shown)
- Run only essential E2E tests on main branch or nightly schedule

---

## 38.11 CI for Migrations (Advanced Checks)

### 38.11.1 Check for “squash migrations needed” (optional)
Large migration graphs can slow deploys and tests. Squashing can help, but do it
carefully.

### 38.11.2 Ensure migration history is linear enough
In teams, conflicting migrations can happen. CI can catch conflicts with:

```bash
python manage.py makemigrations --check
python manage.py showmigrations
```

If you see conflicts, you may need merge migrations.

---

# 38.12 Hands‑On Lab: Add CI to Your Repo

1. Create `config/settings/ci.py`.
2. Add `.github/workflows/ci.yml`.
3. Ensure:
   - tests pass on CI
   - migrations check passes
   - Postgres-backed tests pass
4. Break something intentionally (e.g., format) and confirm CI fails.
5. Fix and confirm CI passes.

---

## 38.13 Exercises (Do These Before Proceeding)

1. Add a “smoke test” step that runs a management command:
   - `python manage.py check --deploy`
   - `python manage.py healthcheck` (if you created one)
2. Add caching for pip installs in GitHub Actions.
3. Split CI into multiple jobs:
   - lint job
   - test job
   - build job
   - make test job depend on lint
4. Add a nightly scheduled job to run:
   - full test suite
   - dependency audit
   - (optional) E2E tests

---

## 38.14 Chapter Summary

- CI enforces code quality, migrations discipline, and correctness on every change.
- Use Postgres in CI to match production behavior.
- Eager Celery in CI keeps tests deterministic without running workers.
- CD should be staged, include migrations and smoke tests, and be designed for rollbacks.
- Secrets must be managed outside repo with least privilege.

---

Next chapter: **39. Caching, CDN, and Performance in Production**  
We’ll move caching from “dev locmem demos” to real production patterns: Redis
caching, cache headers, CDN integration, compression, ETags, and safe invalidation
strategies.